<a href="https://colab.research.google.com/github/Granero0011/DS3-Predictive-Modeling-Challenge/blob/master/Data_Wrangling_Ideas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add your ideas for data wrangling in a cell below. Add code if you would like or just ideas for features/ways to clean the dataset.




1.   Population around each well, higher pop-density would mean higher wear
2.   Does elevation interact with condition of well
3.   Airpollution? compare polution in the area to the longevity of the well 
4.   Does a certain installer have longer lasting wells? 





let's get a majority <-?

In [0]:
###!!! DONT JUST COPY AND PASTE

def MrClean(df):
    df_t= df
    df_t['gps_height'].replace(0.0, np.nan, inplace=True)
    df_t['population'].replace(0.0, np.nan, inplace=True)
    df_t['amount_tsh'].replace(0.0, np.nan, inplace=True)
    df_t['gps_height'].fillna(df_t.groupby(['region', 'district_code'])['gps_height'].transform('mean'), inplace=True)
    df_t['gps_height'].fillna(df_t.groupby(['region'])['gps_height'].transform('mean'), inplace=True)
    df_t['gps_height'].fillna(df_t['gps_height'].mean(), inplace=True)
    df_t['population'].fillna(df_t.groupby(['region', 'district_code'])['population'].transform('median'), inplace=True)
    df_t['population'].fillna(df_t.groupby(['region'])['population'].transform('median'), inplace=True)
    df_t['population'].fillna(df_t['population'].median(), inplace=True)
    df_t['amount_tsh'].fillna(df_t.groupby(['region', 'district_code'])['amount_tsh'].transform('median'), inplace=True)
    df_t['amount_tsh'].fillna(df_t.groupby(['region'])['amount_tsh'].transform('median'), inplace=True)
    df_t['amount_tsh'].fillna(df_t['amount_tsh'].median(), inplace=True)
    features=['amount_tsh', 'gps_height', 'population']
    scaler = MinMaxScaler(feature_range=(0,20))
    df_t[features] = scaler.fit_transform(df_t[features])
    df_t['longitude'].replace(0.0, np.nan, inplace=True)
    df_t['latitude'].replace(0.0, np.nan, inplace=True)
    df_t['construction_year'].replace(0.0, np.nan, inplace=True)
    df_t['latitude'].fillna(df_t.groupby(['region', 'district_code'])['latitude'].transform('mean'), inplace=True)
    df_t['longitude'].fillna(df_t.groupby(['region', 'district_code'])['longitude'].transform('mean'), inplace=True)
    df_t['longitude'].fillna(df_t.groupby(['region'])['longitude'].transform('mean'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['region', 'district_code'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['region'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['district_code'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t['construction_year'].median(), inplace=True)
    df_t['date_recorded'] = pd.to_datetime(df_t['date_recorded'])
    df_t['years_service'] = df_t.date_recorded.dt.year - df_t.construction_year
   
    garbage=['wpt_name','num_private','subvillage','region_code','recorded_by','management_group',
         'extraction_type_group','extraction_type_class','scheme_name','payment',
        'quality_group','quantity_group','source_type','source_class','waterpoint_type_group',
        'ward','public_meeting','permit','date_recorded','construction_year']
    df_t.drop(garbage,axis=1, inplace=True)
    df_t.waterpoint_type = df_t.waterpoint_type.str.lower()
    df_t.funder = df_t.funder.str.lower()
    df_t.basin = df_t.basin.str.lower()
    df_t.region = df_t.region.str.lower()
    df_t.source = df_t.source.str.lower()
    df_t.lga = df_t.lga.str.lower()
    df_t.management = df_t.management.str.lower()
    df_t.quantity = df_t.quantity.str.lower()
    df_t.water_quality = df_t.water_quality.str.lower()
    df_t.payment_type=df_t.payment_type.str.lower()
    df_t.extraction_type=df_t.extraction_type.str.lower()
    df_t["funder"].fillna("other", inplace=True)
    df_t["scheme_management"].fillna("other", inplace=True)
    df_t["installer"].fillna("other", inplace=True)
    
    #further spacial/location information
    #https://www.kaggle.com/c/sf-crime/discussion/18853
    
    return df_t

In [0]:
def con_matrix_analysis(model, feature_matrix, target_vector):
  
  x = model.predict(feature_matrix)
  y = target_vector
  
  print(classification_report(y, x,
        target_names=['Functional', 'Needs Repair', 'Not-Functional']))

  con_matrix = pd.DataFrame(confusion_matrix(y, x), 
             columns=['Predicted Functional', 'Predicted Needs Repair', 'Predicted Not-Functional'], 
             index=['Actual Functional', 'Actual Needs Repair', 'Actual Not-Functional'])
                            
  sns.heatmap(data=con_matrix, cmap='cool')
  plt.show();
  return con_matrix

In [0]:
def reverse_cardinality_check(n, df):
  """
  Given a cardinality limit (n) and a dataframe this function will search the
  dataframe for features above the cardinality limit, then create a dict
  from the results
  """
  
  feature_list = []
  
  cardinality_value = []
  
  for _ in range(len(df.columns)):
    if len(df[df.columns[_]].value_counts()) > n:
      
      feature_list.append(df.columns[_])
      
      cardinality_value.append(len(df[df.columns[_]].value_counts()))
                               
        
  feature_dict = dict(zip(feature_list, cardinality_value))
  
  return feature_dict